### Multi-class SVM
클래스가 2개 이상인 경우를 다중 클래스 분류(Multi-Class Classification)라 한다. 다중 클래스 분류 문제는 다음과 같이 여러개의 이진 클래스 분류(Binary Class Classification) 문제로 변환하여 해결한다.
<img src="https://www.researchgate.net/profile/Anthony_Fleury/publication/220098164/figure/fig2/AS:305725600485382@1449902071844/Figure-2-Illustration-of-the-SVM-principle-and-of-the-one-versus-one-multiclass.png" width="600" height="600" />
<br>
#### OvR (One-vs-the-Rest)
K 개의 타겟 클래스가 존재하는 경우,
각각의 클래스에 대해 표본이 속하는지 속하지 않는지의 이진 클래스 분류 문제를 풀고
판결 기준값이 가장 큰 클래스를 선택 OneVsRestClassifier 클래스
<img src="https://pythonprogramming.net/static/images/machine-learning/one-vs-rest-svm.png" width="300" height="300" />
<br>
#### OvO (One-Vs-One)
K 개의 타겟 클래스가 존재하는 경우,
이 중 2개의 클래스 조합을 선택하여  K(K−1)/2K(K−1)/2 개의 이진 클래스 분류 문제를 풀고
투표를 통해 가장 많은 표를 얻은 클래스를 선택
실제로는 정규화된 판결 기준값을 이용 OneVsOneClassifier 클래스
<img src="https://pythonprogramming.net/static/images/machine-learning/one-vs-one-svm.png" width="300" height="300" />


## 3가지의 의도로 Text를 분류해보자 (피자주문, 호텔예약, 여행정보)

In [33]:
from konlpy.tag import Mecab
import numpy as np

train_data_list =  {
                    'encode' : ['판교에 오늘 피자 주문해줘','오늘 날짜에 호텔 예약 해줄레','모래 날짜에 판교 여행 정보 알려줘'],
                    'decode' : ['0','1','2']
                   }

embed_type = 'onehot'
encode_length = 8 #문장의 최대 길이 나머지는 Padding로 채움
vector_size = 50
label_size = 3 #Label의 수

def onehot_vectorize(bucket, x):
    #W2V의 Vocab의 해당 index값을 onehot으로 만듬
    np.put(bucket, model.wv.index2word.index(x),1)
    return bucket

def embed(data) : 
    mecab = Mecab('/usr/local/lib/mecab/dic/mecab-ko-dic')
    inputs = []
#   labels = []
    for encode_raw in data['encode'] : 
        encode_raw = mecab.morphs(encode_raw)
        encode_raw = list(map(lambda x : encode_raw[x] if x < len(encode_raw) else '#', range(encode_length)))
        if(embed_type == 'onehot') :
            bucket = np.zeros(vector_size, dtype=float).copy()
            input = np.array(list(map(lambda x : onehot_vectorize(bucket, x) if x in model.wv.index2word else np.zeros(vector_size,dtype=float) , encode_raw)))
        inputs.append(input.flatten())
        print(encode_raw)
#     for decode_raw in data['decode']: 
#         label = np.zeros(label_size, dtype=float)
#         np.put(label, decode_raw, 1)
#         labels.append(label)
    return inputs #labels

#X, y = embed(train_data_list) #Encode와 Decode Data를 X와 y값에 Vector값을 담음
X = embed(train_data_list) #Encode와 Decode Data를 X와 y값에 Vector값을 담음
y = train_data_list['decode']

['판교', '에', '오늘', '피자', '주문', '해', '줘', '#']
['오늘', '날짜', '에', '호텔', '예약', '해', '줄레', '#']
['모래', '날짜', '에', '판교', '여행', '정보', '알려줘', '#']


In [34]:
from gensim.models import word2vec

def train_vector_model(str_buf):
    mecab = Mecab('/usr/local/lib/mecab/dic/mecab-ko-dic')
    str_buf = train_data_list['encode']
    #mecab로 POS Tagging
    pos1 = mecab.pos(''.join(str_buf))
    #문장별로 list로 나눔 마침표등이 존재시 줄바꾸기 (문장이길경우)
    pos2 = ' '.join(list(map(lambda x : '\n' if x[1] in ['SF'] else x[0], pos1))).split('\n')
    #단어구성을 위한 형태소단위 문장 쪼개기 
    morphs = list(map(lambda x : mecab.morphs(x) , pos2))
    model = word2vec.Word2Vec(size=vector_size, window=2, min_count=1)
    model.build_vocab(morphs)
    model.train(morphs)
    return model
# W2V Vector 모델 생성
model = train_vector_model(train_data_list)
print(model)
print(model.wv.index2word) #Word List를 구함

Word2Vec(vocab=15, size=50, alpha=0.025)
['에', '오늘', '날짜', '판교', '해', '주문', '알려줘', '피자', '정보', '줘', '모래', '호텔', '여행', '줄레', '예약']


In [38]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(X, y) 
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
# print(clf.predict([[2, 2]]))
print(clf.predict(X[0]))

['0']


/opt/conda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
